# Cookbook 3: Validate data with GX Core and GX Cloud

This cookbook showcases a sample GX data validation workflow characteristic of validation performed on existing organizational data.

## Imports

This tutorial features the `great_expectations` library.

The `tutorial_code` module contains helper functions used within this notebook and the associated Airflow pipeline.

The `airflow_dags` submodule is included so that you can inspect the code used in the related Airflow DAG directly from this notebook.

In [ ]:
import great_expectations as gx
import great_expectations.expectations as gxe

import tutorial_code as tutorial
import airflow_dags.cookbook2_validate_and_handle_invalid_data as dag

## Verify GX Cloud credentials are defined

This cookbook persists validation results to GX Cloud, and requires that valid GX Cloud organization credentials are provided.

In [ ]:
tutorial.cloud.check_for_gx_cloud_credentials_exist()

```{warning} GX Cloud credential error
If `tutorial.cloud.check_for_gx_cloud_credentials_exist()` rasies a `ValueError` indicating that `GX_CLOUD_ORGANIZATION_ID` or `GX_CLOUD_ACCESS_TOKEN` is undefined, ensure that you have provided your GX Cloud organization id and access token when starting Docker compose.
```